### MNIST Classifier using a pre-trained ResNet model

In this exercise, we use a ResNet-18 model to classify items from the MNIST dataset.

First, we import the necessary modules:

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm